In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from pypdf import PdfReader
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        print(url)
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No Title Found"
        for irrelevant in soup.body("script", "style", "img", "input"):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

class Resume:
    def __init__(self, resume_path):
        self.pdfReader = PdfReader(resume_path)
        self.text = self.pdfReader.pages[0].extract_text()

In [ ]:
system_prompt = "You are a resume checker and have internal knowledge about what the ATS (applicant tracking system) is identifying as needed when observing the uploaded resume"

In [ ]:
def user_prompt(website, resume):
    user_prompt = f"You are looking at a website titled: {website.title}"
    user_prompt += "\nPlease use this website to make key adjustments to the uploaded resume on hand. \
    I am the person looking to be hired by this company and would like to be given at least a fair chance to interview\
    Please suggest necessary changes and improvements in list format -- especially within the Projects or Experience sections.\
    Also, please make your suggestions using Markdown.\n\n" 
    user_prompt += resume.text
    return user_prompt
    

In [ ]:
def messages_for_user(website, resume):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt(website, resume)}
    ]

In [ ]:
def summarize(url):
    website = Website(url)
    resume = Resume('grier_resume.pdf')
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for_user(website, resume)
    )
    return response.choices[0].message.content

In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary('https://search-careers.gm.com/en/jobs/jr-202500570/data-scientist')